# Neptune API tour


## Introduction

This guide will show you how to:

* Install neptune
* Connect Neptune to your script and create the first run
* Log simple metrics to Neptune and explore them in the web app
* Log learning curves, images and model binaries from Keras training and see those in the Neptune app
* Fetch the data you logged to Neptune directly into your notebook and analyze them 

By the end of it, you will execute your first run and see it in Neptune!

## Before you start

This notebook example lets you try out Neptune as an anonymous user, with zero setup.

If you want to see the example logged to your own workspace instead:

  1. Create a Neptune account. [Register &rarr;](https://neptune.ai/register)
  1. Create a Neptune project that you will use for tracking metadata. For instructions, see [Creating a project](https://docs.neptune.ai/setup/creating_project) in the Neptune docs.

## Install Neptune and dependencies

In [ ]:
! pip install -U neptune

## Initialize Neptune

To create a new run for tracking the metadata, you tell Neptune who you are (`api_token`) and where to send the data (`project`).

You can use the default code cell below to create an anonymous run in a public project. **Note**: Public projects are cleaned regularly, so anonymous runs are only stored temporarily.

### Log to your own project instead

Replace the code below with the following:

```python
import neptune
from getpass import getpass

run = neptune.init_run(
    project="workspace-name/project-name",  # replace with your own (see instructions below)
    api_token=getpass("Enter your Neptune API token: "),
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
)
```

To find your API token and full project name:

1. [Log in to Neptune](https://app.neptune.ai/).
1. In the bottom-left corner, expand your user menu and select **Get your API token**.
1. The workspace name is displayed in the top-left corner of the app. 

    To copy the project path, in the top-right corner, open the settings menu and select **Properties**.

For more help, see [Setting Neptune credentials](https://docs.neptune.ai/setup/setting_credentials) in the Neptune docs.

In [ ]:
import neptune

run = neptune.init_run(
    project="common/quickstarts",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    capture_hardware_metrics=True,
    capture_stderr=True,
    capture_stdout=True,
)  # Hardware metrics, stderr, and stdout are not captured by default in interactive kernels

**To open the run in the Neptune web app, click the link that appeared in the cell output.**

We'll use the `run` object we just created to log metadata. You'll see the metadata appear in the app.

## Basic Example

Lets start with something super simple.

You will:
* log parameters
* log a training and validation scores
* log loss
* add name to the run
* append two tags

In [ ]:
params = {"learning_rate": 0.1}

# log params
run["parameters"] = params

# log name and append tags
run["sys/name"] = "basic-colab-example"
run["sys/tags"].add(["colab", "intro"])

# log loss during training
for epoch in range(100):
    run["train/loss"].append(0.99**epoch)

# log train and validation scores
run["train/accuracy"] = 0.95
run["valid/accuracy"] = 0.93

Runs can be viewed as dictionary-like structures - **namespaces** - that you can define in your code. You can apply hierarchical structure to your metadata that will be reflected in the web app as well. Thanks to this you can easily organize your metadata in a way you feel is most convenient.

There is one special namespace: **system namespace**, denoted `sys`. You can use it to add name and tags to the run.

Try to change the values and re-execute to see your runs appear in the dashboard.

All `basic-colab-example` runs are grouped in [this dashboard view](https://app.neptune.ai/o/common/org/quickstarts/experiments?split=tbl&dash=charts&viewId=96fe9215-15ff-4bf8-9096-9d06a99a8b36).

![colab-runs](https://neptune.ai/wp-content/uploads/Neptune_API_Tour_colab_runs.png)

There are many other things that you can log to neptune:

* Images and charts
* Artifacts like model weights or results
* Text values
* Hardware consumption
* Code snapshots
* and more

You can go and see all that in the [documentation](https://docs.neptune.ai/user-guides/logging-and-managing-runs-results/logging-runs-data), and also check out the next example to see some of those.


## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Keras classification example [Advanced]

Install and import your machine learning libraries

In [ ]:
! pip install scikit-plot tensorflow

Get the data

In [ ]:
import tensorflow as tf

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

Prepare the model

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation="relu"),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation="softmax"),
    ]
)

model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

To log metrics after every batch and epoch let's create `NeptuneLogger` callback:

In [ ]:
from tensorflow.keras.callbacks import Callback


class NeptuneLogger(Callback):
    def on_batch_end(self, batch, logs=None):
        if logs is None:
            logs = {}
        for log_name, log_value in logs.items():
            run[f"batch/{log_name}"].append(log_value)

    def on_epoch_end(self, epoch, logs=None):
        if logs is None:
            logs = {}
        for log_name, log_value in logs.items():
            run[f"epoch/{log_name}"].append(log_value)

Now we simply need to create a run.
I will tag it with the name `advanced` and log hyperparameters `epoch_nr` and `batch_size`: 

In [ ]:
EPOCH_NR = 5
BATCH_SIZE = 32

run = neptune.init_run(project="common/quickstarts", api_token=neptune.ANONYMOUS_API_TOKEN)

# log params
run["parameters/epoch_nr"] = EPOCH_NR
run["parameters/batch_size"] = BATCH_SIZE

# log name and append tag
run["sys/name"] = "keras-metrics"
run["sys/tags"].add("advanced")

Now we pass our `NeptuneLogger` as keras callback and thats it.

In [ ]:
history = model.fit(
    x=x_train,
    y=y_train,
    epochs=EPOCH_NR,
    batch_size=BATCH_SIZE,
    validation_data=(x_test, y_test),
    callbacks=[NeptuneLogger()],
)

You can click on the run link above and monitor your learning curves as it is training!

![advanced](https://neptune.ai/wp-content/uploads/charts-in-neptune.png)


Great thing is, you can log more things if you need to during or after the training is finished.


For example, let's calculate some additional metrics on test data and log them.

In [ ]:
import numpy as np

y_test_pred = np.asarray(model.predict(x_test))
y_test_pred_class = np.argmax(y_test_pred, axis=1)

In [ ]:
from sklearn.metrics import f1_score

run["test/f1"] = f1_score(y_test, y_test_pred_class, average="micro")

We can log diagnostic charts like confusion matrix or ROC AUC curve.

In [ ]:
import matplotlib.pyplot as plt
from scikitplot.metrics import plot_confusion_matrix, plot_roc

fig, ax = plt.subplots(figsize=(16, 12))
plot_confusion_matrix(y_test, y_test_pred_class, ax=ax)
run["diagnostic_charts"].append(fig)

fig, ax = plt.subplots(figsize=(16, 12))
plot_roc(y_test, y_test_pred, ax=ax)
run["diagnostic_charts"].append(fig)

![Saved charts](https://neptune.ai/wp-content/uploads/Neptune_API_Tour_saved_charts.png)

We can also log model weights to Neptune.

In [ ]:
model.save("my_model.h5")
run["model"].upload("my_model.h5")

![Model](https://neptune.ai/wp-content/uploads/model-metadata-in-neptune.png)

With that you can share models with your teammates easily.

You can play around and execute this run with different parameters and see results and compare them.

A cool thing is, once things are logged to Neptune you can access them from wherever you want.
Let me show you. 

## Access data you logged programatically 

## Getting the project's leaderboard

You can use the project object to get aggregate data of all your runs. Here, fetch only runs with tag 'advanced'.

In [ ]:
my_project = neptune.init_project(
    project="common/quickstarts", api_token=neptune.ANONYMOUS_API_TOKEN, mode="read-only"
)
run_df = my_project.fetch_runs_table(tag=["advanced"]).to_pandas()
run_df.head()

## Stop logging

Once you are done logging, stop tracking the run.

In [ ]:
run.stop()

## Getting the run's metadata

Neptune lets you fetch whatever you logged to it directly to your notebooks and scripts.

In [1]:
import neptune

In [2]:
run = neptune.init_run(
    project="common/quickstarts",
    api_token=neptune.ANONYMOUS_API_TOKEN,
    with_id="QUI-88292",
    mode="read-only",
)

https://app.neptune.ai/common/quickstarts/e/QUI-88292


Now that your run is *fetched* you can download the metadata.

In [3]:
batch_size = run["parameters/batch_size"].fetch()
last_batch_acc = run["batch/accuracy"].fetch_last()
print(f"batch_size: {batch_size}")
print(f"last_batch_acc: {last_batch_acc}")

batch_size: 32
last_batch_acc: 0.9361500144004822


You can even download artifacts from that run if you want to:



In [4]:
run["model"].download()

In [5]:
! ls ./

Neptune_API_Tour.ipynb
model.h5
my_model.h5


## Stop logging

Once you are done logging, stop tracking the run.

In [6]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Explore the metadata in the Neptune app:
https://app.neptune.ai/common/quickstarts/e/QUI-88292/metadata


## Learn more about Neptune

Read about other Neptune features, create your free account and start logging!

[Go to Neptune](https://neptune.ai/register)